In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F

In [2]:

spark = (
    SparkSession.builder.appName("ToParquetLeagueOfData")
    .master("spark://spark-master:7077")
    .config(
        "spark.jars.packages",
        "com.amazonaws:aws-java-sdk:1.11.563,com.amazonaws:aws-java-sdk-bundle:1.11.874,org.apache.hadoop:hadoop-aws:3.3.2,io.delta:delta-core_2.12:2.1.0"
    )
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.hadoop.fs.s3a.multipart.size", "838860800")
    .config(
        "spark.hadoop.fs.s3a.aws.credentials.provider",
        "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider",
    )
    .config("spark.hadoop.fs.s3a.path.style.access", True)
    .getOrCreate()
)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
com.amazonaws#aws-java-sdk added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fbef6017-1a5e-4b08-a66f-adcdcccba44d;1.0
	confs: [default]
	found com.amazonaws#aws-java-sdk;1.11.563 in central
	found com.amazonaws#aws-java-sdk-ioteventsdata;1.11.563 in central
	found com.amazonaws#aws-java-sdk-core;1.11.563 in central
	found commons-logging#commons-logging;1.1.3 in central
	found org.apache.httpcomponents#httpclient;4.5.5 in central
	found org.apache.httpcomponents#httpcore;4.4.9 in central
	found commons-codec#commons-codec;1.10 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found com.fasterxml.jackson.core#jackson-databind;2.6.7.2 in central
	found com.fasterxml.jackson.

23/01/06 18:55:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark.sparkContext.setLogLevel("ERROR")
sc = spark
hadoop_conf = sc._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("fs.s3a.endpoint", "http://minio:9000")
hadoop_conf.set("fs.s3a.access.key", "cDmBsEallXwr8QAe")
hadoop_conf.set("fs.s3a.secret.key", "1L34sxo5OqJlTJF8co1UiHsCabdZ96bH")
hadoop_conf.set("fs.s3a.path.style.access", "True")

In [5]:
time_line_df = spark.read.json("s3a://league-of-data-bronze/matchs/timeline")

In [7]:
time_line_df.printSchema()

root
 |-- info: struct (nullable = true)
 |    |-- frameInterval: long (nullable = true)
 |    |-- frames: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- events: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- actualStartTime: long (nullable = true)
 |    |    |    |    |    |-- afterId: long (nullable = true)
 |    |    |    |    |    |-- assistingParticipantIds: array (nullable = true)
 |    |    |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |    |    |-- beforeId: long (nullable = true)
 |    |    |    |    |    |-- bounty: long (nullable = true)
 |    |    |    |    |    |-- buildingType: string (nullable = true)
 |    |    |    |    |    |-- creatorId: long (nullable = true)
 |    |    |    |    |    |-- gameId: long (nullable = true)
 |    |    |    |    |    |-- goldGain: long (nullable = true)
 |    |    |    |    |    |-- itemId: 

In [22]:
time_line_info_df = time_line_df.select(col("info.*"), col("match"), col("extracted_at"))
time_line_info_df.show()

+-------------+--------------------+----------+--------------------+--------------+------------+
|frameInterval|              frames|    gameId|        participants|         match|extracted_at|
+-------------+--------------------+----------+--------------------+--------------+------------+
|        60000|[{[{null, null, n...|2649901305|[{1, wkyR5XeIB7YV...|BR1_2649901305|  2023-01-05|
|        60000|[{[{null, null, n...|2649901305|[{1, wkyR5XeIB7YV...|BR1_2649901305|  2023-01-06|
|        60000|[{[{null, null, n...|2654852861|[{1, W2oYyoRE3lL5...|BR1_2654852861|  2023-01-06|
|        60000|[{[{null, null, n...|2654852861|[{1, W2oYyoRE3lL5...|BR1_2654852861|  2023-01-05|
|        60000|[{[{null, null, n...|2614946619|[{1, XUbwp34tgkpl...|BR1_2614946619|  2023-01-05|
|        60000|[{[{null, null, n...|2614946619|[{1, XUbwp34tgkpl...|BR1_2614946619|  2023-01-06|
|        60000|[{[{null, null, n...|2623878701|[{1, bRcp98C8U4Nm...|BR1_2623878701|  2023-01-06|
|        60000|[{[{null, null,

In [44]:
time_line_frames.select(col("frames.participantFrames.1")).show()

+--------------------+
|                   1|
+--------------------+
|{{0, 0, 30, 0, 0,...|
|{{0, 0, 30, 0, 0,...|
|{{0, 0, 30, 0, 0,...|
|{{0, 0, 33, 0, 0,...|
|{{0, 0, 36, 0, 0,...|
|{{0, 0, 36, 0, 0,...|
|{{0, 0, 40, 0, 0,...|
|{{0, 0, 44, 0, 0,...|
|{{0, 0, 44, 0, 0,...|
|{{0, 0, 48, 0, 0,...|
|{{0, 0, 48, 0, 0,...|
|{{0, 0, 48, 0, 0,...|
|{{0, 0, 52, 0, 0,...|
|{{0, 0, 52, 0, 0,...|
|{{0, 0, 56, 0, 0,...|
|{{0, 0, 56, 0, 0,...|
|{{0, 0, 61, 0, 0,...|
|{{0, 0, 61, 0, 0,...|
|{{0, 0, 65, 0, 0,...|
|{{0, 0, 65, 0, 0,...|
+--------------------+
only showing top 20 rows

